In [1]:
import pandas as pd, numpy as np
from rdkit import Chem

In [2]:
from scipy import stats

In [3]:
import pandas as pd, numpy as np, re, os
from evaluation import Evaluator, compilerPatterns, ResultsFromMeanStdDF
from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 12})

In [5]:
paths = [ os.path.join(dataset.name, experiment.name) 
         for dataset in os.scandir('experiments') if dataset.is_dir() and 'grisoni' in dataset.name 
         for experiment in os.scandir(dataset.path)  ]

In [6]:
evaluators = [Evaluator(path, novel=True) for path in paths]

In [ ]:
scaffDFs=pd.concat( [evaluator.getScaffoldResults() for evaluator in evaluators if not evaluator.gens.empty], ignore_index=True)

In [8]:
scaffDFs['unique%']=scaffDFs['unique']/scaffDFs['total']
scaffDFs['novel%']=scaffDFs['novel']/scaffDFs['unique']
scaffDFs['chiral%']=scaffDFs['chiral']/scaffDFs['novel']

In [9]:
scaffGrp=scaffDFs.fillna(-1).groupby(compilerPatterns.index.drop('fold').tolist(),as_index=True, dropna=False)

/tmp/ipykernel_1141590/2999511724.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  scaffGrp=scaffDFs.fillna(-1).groupby(compilerPatterns.index.drop('fold').tolist(),as_index=True, dropna=False)


In [10]:
scaffStats=scaffGrp.describe().loc(axis=1)[ ( slice(None),['mean','std'] ) ].drop(columns='fold')

In [11]:
scaffStats=ResultsFromMeanStdDF(scaffStats)

In [12]:
scaffStats.query(' ( aug==5 and hl==2 and hu==512 and bs==256 ) or ( aug!=5 and hl==2 and hu==512 and bs==512 ) ')

total  \
dataset aug notation   hl hu  es   lr    bs  amount epoch T                   
grisoni -1  SELFIES    2  512 -1   0.001 512 6000   10    1.0  5964.0 ± 6.0   
            SMILES     2  512 -1   0.001 512 6000   10    1.0  5976.0 ± 8.0   
            fragSMILES 2  512  300 0.001 512 6000   16    1.0  5985.0 ± 8.0   
         5  SELFIES    2  512 -1   0.001 256 6000   3     1.0  5978.0 ± 6.0   
            SMILES     2  512 -1   0.001 256 6000   3     1.0  5976.0 ± 5.0   
            fragSMILES 2  512  300 0.001 256 6000   4     1.0  5978.0 ± 7.0   

                                                                      unique  \
dataset aug notation   hl hu  es   lr    bs  amount epoch T                    
grisoni -1  SELFIES    2  512 -1   0.001 512 6000   10    1.0  5570.0 ± 40.0   
            SMILES     2  512 -1   0.001 512 6000   10    1.0  5490.0 ± 80.0   
            fragSMILES 2  512  300 0.001 512 6000   16    1.0  5500.0 ± 50.0   
         5  SELFIES    2  512 -1   0.001 256 6000   3     1.0  5620.0 ± 40.0   
            SMILES     2  512 -1   0.001 256 6000   3     1.0  5610.0 ± 70.0   
            fragSMILES 2  512  300 0.001 256 6000   4     1.0  5510.0 ± 80.0   

                                                                        novel  \
dataset aug notation   hl hu  es   lr    bs  amount epoch T                     
grisoni -1  SELFIES    2  512 -1   0.001 512 6000   10    1.0   5160.0 ± 70.0   
            SMILES     2  512 -1   0.001 512 6000   10    1.0  4600.0 ± 100.0   
            fragSMILES 2  512  300 0.001 512 6000   16    1.0   4440.0 ± 70.0   
         5  SELFIES    2  512 -1   0.001 256 6000   3     1.0   5240.0 ± 60.0   
            SMILES     2  512 -1   0.001 256 6000   3     1.0  5000.0 ± 100.0   
            fragSMILES 2  512  300 0.001 256 6000   4     1.0  4400.0 ± 100.0   

                                                                       chiral  \
dataset aug notation   hl hu  es   lr    bs  amount epoch T                     
grisoni -1  SELFIES    2  512 -1   0.001 512 6000   10    1.0    760.0 ± 40.0   
            SMILES     2  512 -1   0.001 512 6000   10    1.0    820.0 ± 40.0   
            fragSMILES 2  512  300 0.001 512 6000   16    1.0    750.0 ± 50.0   
         5  SELFIES    2  512 -1   0.001 256 6000   3     1.0    870.0 ± 60.0   
            SMILES     2  512 -1   0.001 256 6000   3     1.0  1000.0 ± 100.0   
            fragSMILES 2  512  300 0.001 256 6000   4     1.0    830.0 ± 80.0   

                                                                     unique%  \
dataset aug notation   hl hu  es   lr    bs  amount epoch T                    
grisoni -1  SELFIES    2  512 -1   0.001 512 6000   10    1.0  0.934 ± 0.007   
            SMILES     2  512 -1   0.001 512 6000   10    1.0    0.92 ± 0.01   
            fragSMILES 2  512  300 0.001 512 6000   16    1.0  0.918 ± 0.007   
         5  SELFIES    2  512 -1   0.001 256 6000   3     1.0  0.941 ± 0.006   
            SMILES     2  512 -1   0.001 256 6000   3     1.0    0.94 ± 0.01   
            fragSMILES 2  512  300 0.001 256 6000   4     1.0    0.92 ± 0.01   

                                                                      novel%  \
dataset aug notation   hl hu  es   lr    bs  amount epoch T                    
grisoni -1  SELFIES    2  512 -1   0.001 512 6000   10    1.0  0.925 ± 0.006   
            SMILES     2  512 -1   0.001 512 6000   10    1.0  0.838 ± 0.009   
            fragSMILES 2  512  300 0.001 512 6000   16    1.0  0.809 ± 0.007   
         5  SELFIES    2  512 -1   0.001 256 6000   3     1.0  0.931 ± 0.004   
            SMILES     2  512 -1   0.001 256 6000   3     1.0     0.9 ± 0.01   
            fragSMILES 2  512  300 0.001 256 6000   4     1.0    0.81 ± 0.01   

                                                                     chiral%  
dataset aug notation   hl hu  es   lr    bs  amount epoch T                   
grisoni -1  SELFIES    2  512 -1   0.001 512 6000   10    

In [ ]:
scaffStats.to_csv('results/08_grisoniScaffMetrics.csv')

In [13]:
params=[2,512,-1,0.001,512,6000,10,1.0]
paramsFrag = [2,512,300,0.001,512,6000,16,1.0]

In [14]:
params5=[2,512,-1,0.001,256,6000,3,1.0]
paramsFrag5 = [2,512,300,0.001,256,6000,4,1.0]

In [15]:
genPvals = pd.DataFrame.from_dict(
                {notation+'x1': 
                {metric:stats.ttest_ind(
                scaffGrp.get_group(('grisoni',-1,notation,*params))[metric],
                scaffGrp.get_group(('grisoni',-1,'fragSMILES',*paramsFrag))[metric],
                ).pvalue for metric in scaffStats.columns} for notation in ['SMILES','SELFIES']
                } |
                {notation+'x5': 
                {metric:stats.ttest_ind(
                scaffGrp.get_group(('grisoni',5,notation,*params5))[metric],
                scaffGrp.get_group(('grisoni',5,'fragSMILES',*paramsFrag5))[metric],
                ).pvalue for metric in scaffStats.columns} for notation in ['SMILES','SELFIES']
                }
                , orient='index'
            )

In [16]:
genPvals

,total,unique,novel,chiral,unique%,novel%,chiral%
SMILESx1,0.122287,0.883691,2.905847e-02,0.042347,0.966772,3.886867e-04,0.125945
SELFIESx1,0.001574,0.034050,2.212652e-07,0.642110,0.007221,3.270683e-09,0.005719
SMILESx5,0.544191,0.070134,1.120501e-04,0.116053,0.064472,4.878120e-06,0.846064
SELFIESx5,1.000000,0.017568,2.491563e-06,0.417392,0.017827,9.042331e-08,0.042876


In [17]:
genPvals.map(lambda x: x<0.05)

,total,unique,novel,chiral,unique%,novel%,chiral%
SMILESx1,False,False,True,True,False,True,False
SELFIESx1,True,True,True,False,True,True,True
SMILESx5,False,False,True,False,False,True,False
SELFIESx5,False,True,True,False,True,True,True


## Count how many scaffolds are novel cause of new cyclic fragments

In [18]:
from ast import literal_eval
notations = ['SELFIES','SMILES','fragSMILES']
def _converting(cell):
    if cell.startswith('(') :
        return literal_eval(cell)

In [19]:
from processer import fragmentSmilesList

In [54]:
def checkScaffoldFragments(scaffolds, vocab):

    def checkNewFrag(smi, frags):
        if pd.isnull(frags):
            if smi in vocab:
                return False
            return True

        mask = np.in1d(frags, vocab)

        newFrags = np.where(~mask)[0]

        if len(newFrags)==0:
            return False
        
        ## If u want track which new fragments are
        # return np.array(frags)[newFrags]
        return True
    
    # try:
    fragScaff=fragmentSmilesList(scaffolds)
    maskNewFrags=fragScaff.apply(lambda x: checkNewFrag(x.smiles, x.tokens), axis=1)

    return maskNewFrags.sum()
    # except:
    #     return -1

In [21]:
vocabs=[]
for fold in range(5):
    train_data = pd.read_csv(f'experiments/data/grisoni_train_{fold}.tar.xz', usecols = ['fragsmiles'], compression="xz",converters={'fragsmiles': _converting} ).squeeze()
    vocabs.append(np.unique(np.concatenate(train_data.apply(np.array))))

In [22]:
def countNewScaffCauseFrags(evaluator):
    results = evaluator.gens.copy()

    results['inducedNewFrags']=results.apply(lambda x: checkScaffoldFragments(x.sampled.novelScaff, vocabs[x.fold]), axis=1)

    results.drop(columns='sampled', inplace=True)
    return results.sort_values(['amount','fold','epoch'])

In [ ]:
idxsEvaluators = np.unique(scaffDFs.query(' ( aug==5 and hl==2 and hu==512 and bs==256 and notation!="fragSMILES" ) or ( aug!=5 and hl==2 and hu==512 and bs==512 and notation!="fragSMILES") ').index//5)

In [ ]:
inducedNewFrags=pd.concat( [ countNewScaffCauseFrags(evaluators[idx]) for idx in idxsEvaluators], ignore_index=True)

In [84]:
induced = scaffDFs.query(' ( aug==5 and hl==2 and hu==512 and bs==256 and notation!="fragSMILES" ) or ( aug!=5 and hl==2 and hu==512 and bs==512 and notation!="fragSMILES" ) ').merge(inducedNewFrags, 
on=compilerPatterns.index.tolist())

In [86]:
induced['inducedNewFrags%']=induced['inducedNewFrags']/induced['novel']

In [87]:
inducedGrp=induced.fillna(-1).groupby(compilerPatterns.index.drop('fold').tolist(),as_index=True, dropna=False)

/tmp/ipykernel_1141590/238393982.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  inducedGrp=induced.fillna(-1).groupby(compilerPatterns.index.drop('fold').tolist(),as_index=True, dropna=False)


In [88]:
inducedStats=inducedGrp.describe().loc(axis=1)[ ( slice(None),['mean','std'] ) ].drop(columns='fold')

In [89]:
inducedStats=ResultsFromMeanStdDF(inducedStats)

In [ ]:
inducedStats